# DrugBAN Running Demo | [Paper](https://doi.org/10.1038/s42256-022-00605-1) | [Repo](https://github.com/peizhenbai/DrugBAN)

| [Open In Colab](https://colab.research.google.com/github/pz-white/DrugBAN/blob/main/drugban_demo.ipynb) (click `Runtime` → `Run all (Ctrl+F9)` |

This is a code demo of DrugBAN framework for drug-target interaction prediction. It takes about 3 minutes to run the whole pipeline.

## Setup

The first few blocks of code are necessary to set up the notebook execution environment. This checks if the notebook is running on Google Colab and installs required packages.

In [ ]:
if 'google.colab' in str(get_ipython()):
    print('Running on CoLab')
    !pip uninstall --yes yellowbrick
    !pip install -U -q psutil
    !pip install dgl dgllife
    !pip install rdkit-pypi
    !pip install PrettyTable yacs
    !git clone https://github.com/pz-white/DrugBAN.git
    %cd DrugBAN
else:
    print('Not running on CoLab')

## Import required modules.

In [ ]:
from models import DrugBAN
from time import time
from utils import set_seed, graph_collate_func, mkdir
from configs import get_cfg_defaults
from dataloader import DTIDataset, MultiDataLoader
from torch.utils.data import DataLoader
from trainer import Trainer
from domain_adaptator import Discriminator
import torch
import argparse
import warnings, os
import pandas as pd

## Configuration

The customized configuration used in this demo is stored in `configs/DrugBAN_Demo.yaml`, this file overwrites defaults in `config.py` where a value is specified.

For saving time to run a whole pipeline in this demo, we sample small subsets from the original BindingDB dataset, which is located at `datasets/bindingdb_sample`

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
cfg_path = "./configs/DrugBAN_Demo.yaml"
data = "bindingdb_sample"
comet_support = False

cfg = get_cfg_defaults()
cfg.merge_from_file(cfg_path)
cfg.freeze()

torch.cuda.empty_cache()
warnings.filterwarnings("ignore")
set_seed(cfg.SOLVER.SEED)
mkdir(cfg.RESULT.OUTPUT_DIR)
experiment = None
print(f"Config yaml: {cfg_path}")
print(f"Running on: {device}")
print(f"Hyperparameters:")
dict(cfg)

## Data Loader

The train/valid/test datasets are specified using the `DTIDataset()` function and loaded using the `DataLoader()` function.

In [ ]:
dataFolder = f'./datasets/{data}'

train_path = os.path.join(dataFolder, 'train.csv')
val_path = os.path.join(dataFolder, "val.csv")
test_path = os.path.join(dataFolder, "test.csv")
df_train = pd.read_csv(train_path)
df_val = pd.read_csv(val_path)
df_test = pd.read_csv(test_path)

train_dataset = DTIDataset(df_train.index.values, df_train)
val_dataset = DTIDataset(df_val.index.values, df_val)
test_dataset = DTIDataset(df_test.index.values, df_test)

params = {'batch_size': cfg.SOLVER.BATCH_SIZE, 'shuffle': True, 'num_workers': cfg.SOLVER.NUM_WORKERS, 'drop_last': True, 'collate_fn': graph_collate_func}
training_generator = DataLoader(train_dataset, **params)
params['shuffle'] = False
params['drop_last'] = False
val_generator = DataLoader(val_dataset, **params)
test_generator = DataLoader(test_dataset, **params)

## Setup Model and Optimizer

Here, we use the previously defined configuration to set up the model and optimizer we will subsequently train.


In [ ]:
model = DrugBAN(**cfg).to(device)
opt = torch.optim.Adam(model.parameters(), lr=cfg.SOLVER.LR)
if torch.cuda.is_available():
  torch.backends.cudnn.benchmark = True

## Model Training and Test Optimized Model

Optimize model parameters using the trainer and check test performance.

In [ ]:
trainer = Trainer(model, opt, device, training_generator, val_generator, test_generator, opt_da=None, discriminator=None, experiment=experiment, **cfg)
result = trainer.train()
with open(os.path.join(cfg.RESULT.OUTPUT_DIR, "model_architecture.txt"), "w") as wf:
    wf.write(str(model))
print(f"Directory for saving result: {cfg.RESULT.OUTPUT_DIR}")

## Expected Output

Awesome! You complete all demo steps and should get output like the following. Please note that these numbers might be different due to the update of environment setup on colab.

```
Training at Epoch 1 with training loss 0.7483742804754347
Validation at Epoch 1 with validation loss 0.6943950802087784  AUROC 0.6544117647058824 AUPRC 0.44206349206349205
Test at Best Model of Epoch 1 with test loss 0.6565468311309814  AUROC 0.4245614035087719 AUPRC 0.4018830588082055 Sensitivity 0.0 Specificity 1.0 Accuracy 0.3877551020408163 Thred_optim 0.42230069637298584
Directory for saving result: ./result/demo
```

Finally, the output result is saved in the colab temporary directory: `DrugBAN/result/demo`. You can access it by clicking `Files` tab on the left side of colab interface.